In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
import itertools
import math
import re
import lightgbm as lgb

In [21]:
idx = pd.IndexSlice

In [28]:
holidays = pd.read_csv('/content/holidays_events.csv')
oil = pd.read_csv('/content/oil.csv')
stores = pd.read_csv('/content/stores.csv')
transactions = pd.read_csv('/content/transactions.csv')
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [29]:
train['test'] = 0
test['test'] = 1

data = pd.concat([train, test], axis=0)

In [30]:
data = data.merge(holidays, on='date', how='left')
data= data.merge(stores, on='store_nbr', how='left')
data= data.merge(oil, on='date', how='left')
data= data.merge(transactions, on=['date', 'store_nbr'], how='left')
data = data.set_index(['store_nbr', 'date', 'family'])
data = data.drop(index='2013-01-01', level=1)
data

id   sales  onpromotion  \
store_nbr date       family                                                     
1.0       2013-01-02 AUTOMOTIVE                     1782     2.0          0.0   
                     BABY CARE                      1783     0.0          0.0   
                     BEAUTY                         1784     2.0          0.0   
                     BEVERAGES                      1785  1091.0          0.0   
                     BOOKS                          1786     0.0          0.0   
...                                                  ...     ...          ...   
9.0       2017-08-31 POULTRY                     3029395     NaN          1.0   
                     PREPARED FOODS              3029396     NaN          0.0   
                     PRODUCE                     3029397     NaN          1.0   
                     SCHOOL AND OFFICE SUPPLIES  3029398     NaN          9.0   
                     SEAFOOD                     3029399     NaN          0.0   

                                                 test type_x locale  \
store_nbr date       family                                           
1.0       2013-01-02 AUTOMOTIVE                     0    NaN    NaN   
                     BABY CARE                      0    NaN    NaN   
                     BEAUTY                         0    NaN    NaN   
                     BEVERAGES                      0    NaN    NaN   
                     BOOKS                          0    NaN    NaN   
...                                               ...    ...    ...   
9.0       2017-08-31 POULTRY                        1    NaN    NaN   
                     PREPARED FOODS                 1    NaN    NaN   
                     PRODUCE                        1    NaN    NaN   
                     SCHOOL AND OFFICE SUPPLIES     1    NaN    NaN   
                     SEAFOOD                        1    NaN    NaN   

                                                locale_name description  \
store_nbr date       family                                               
1.0       2013-01-02 AUTOMOTIVE                         NaN         NaN   
                     BABY CARE                          NaN         NaN   
                     BEAUTY                             NaN         NaN   
                     BEVERAGES                          NaN         NaN   
                     BOOKS                              NaN         NaN   
...                                                     ...         ...   
9.0       2017-08-31 POULTRY                            NaN         NaN   
                     PREPARED FOODS                     NaN         NaN   
                     PRODUCE                            NaN         NaN   
                     SCHOOL AND OFFICE SUPPLIES         NaN         NaN   
                     SEAFOOD                            NaN         NaN   

                                                transferred   city      state  \
store_nbr date       family                                                     
1.0       2013-01-02 AUTOMOTIVE                         NaN  Quito  Pichincha   
                     BABY CARE                          NaN  Quito  Pichincha   
                     BEAUTY                             NaN  Quito  Pichincha   
                     BEVERAGES                          NaN  Quito  Pichincha   
                     BOOKS                              NaN  Quito  Pichincha   
...                                                     ...    ...        ...   
9.0       2017-08-31 POULTRY                            NaN  Quito  Pichincha   
                     PREPARED FOODS                     NaN  Quito  Pichincha   
                     PRODUCE                            NaN  Quito  Pichincha   
                     SCHOOL AND OFFICE SUPPLIES         NaN  Quito  Pichincha   
                     SEAFOOD                            NaN  Quito  Pichincha   

                                              

In [31]:
data_ = data.copy().reset_index()
train = data_[data_['test'] == 0]
test = data_[data_['test'] == 1]
train['date'] = pd.to_datetime(train["date"])
train['day_of_week'] = train['date'].dt.day_of_week
train['day_of_year'] = train['date'].dt.dayofyear
train['day_of_month'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year

In [32]:
grouping_columns = ['year', 'quarter', 'month', 'day_of_week', 'day_of_year', 'day_of_month']
for ind, column in enumerate(grouping_columns):
    grouped_data = train.groupby(column)['sales'].sum()
    grouped_data = pd.DataFrame(grouped_data).reset_index()

In [33]:
data_ = data.copy().reset_index()
grouped_data = data_.groupby(['store_nbr', 'family'])

In [34]:
alphas = [0.95, 0.8, 0.65, 0.5]
lags =[1,7,30]
for a in alphas:
    for i in lags:
        data_[f'sales_lag_{i}_alpha_{a}'] = np.log1p(grouped_data['sales'].transform(lambda x: x.shift(i).ewm(alpha=a, min_periods=1).mean()))

data_['sales_lag_7_alpha_0.5'].describe()

count    1.995891e+06
mean     2.706258e+00
std      2.686858e+00
min      0.000000e+00
25%      0.000000e+00
50%      2.021562e+00
75%      5.130160e+00
max      1.024314e+01
Name: sales_lag_7_alpha_0.5, dtype: float64

In [35]:
data_['date'] = pd.to_datetime(data_["date"])
data_['day_of_week'] = data_['date'].dt.day_of_week
data_['day_of_year'] = data_['date'].dt.dayofyear
data_['day_of_month'] = data_['date'].dt.day
data_['month'] = data_['date'].dt.month
data_['quarter'] = data_['date'].dt.quarter
data_['year'] = data_['date'].dt.year

In [36]:
data_['onpromotion'] = data_['onpromotion'].apply(lambda x: x > 0)

In [37]:
sales_lag_columns = list(data_.filter(like="lag").columns)

In [39]:
training_percentage = 0.8
testing_percentage = 0.2
to_dummies = ['day_of_week', 'day_of_month', 'month', 'quarter', 'year', 'store_nbr', 'type_y', 'cluster', 'family', 'onpromotion', 'type_x',
       'locale', 'locale_name', 'city', 'state']
X = data_.loc[:, [ 'day_of_week', 'day_of_month', 'month', 'quarter', 'year', 'store_nbr', 'type_y', 'cluster', 'family', 'onpromotion', 'type_x',
       'locale', 'locale_name',  'city', 'state', 'test', 'sales', 'id']+ sales_lag_columns]
X[to_dummies] = X[to_dummies].astype('category')
data_train = X[X['test'] == 0]
data_test = X[X['test'] == 1]
n = len(data_train)
training_start = 0
training_end = math.floor(n * training_percentage)
validation_start = training_end
validation_end = n
X_train = data_train.loc[training_start:training_end, :].drop(['test', 'sales', 'id'],  axis=1)
y_train = data_train.loc[training_start:training_end, 'sales']
X_val = data_train.loc[validation_start:validation_end,  :].drop(['test', 'sales', 'id'],  axis=1)
y_val = data_train.loc[validation_start:validation_end, 'sales']
X_test = data_test.loc[:, ].drop(['test', 'sales', 'id'],  axis=1)

In [40]:
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_val = X_val.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [41]:
hyper_params = {'task': 'train','boosting_type': 'gbdt','objective': 'regression','metric': ['l1','l2'],'learning_rate': 0.1,
'feature_fraction': 0.9,'bagging_fraction': 0.7,'bagging_freq': 10,'verbose': 0,"max_depth": 50,"num_leaves": 128,"max_bin": 512}
gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='l1')

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_fr

LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              max_bin=512, max_depth=50, metric=['l1', 'l2'], num_leaves=128,
              objective='regression', task='train', verbose=0)

In [42]:
y_pred = gbm.predict(X_val)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


In [44]:
results = pd.concat([y_val.reset_index(drop=True), pd.Series(y_pred)], axis=1).rename(columns={'sales': 'y_val', 0: 'y_pred'})
results['y_pred'] = results['y_pred'].clip(0)
results = results[results['y_val'] > 10]
results

,y_val,y_pred
0,47.000,17.801738
1,27.000,39.312913
3,532.000,416.635263
6,53.000,27.378440
9,423.135,386.038867
...,...,...
395961,275.000,290.576536
395962,33.000,52.169517
395963,23.000,86.715012
395964,2699.000,2448.018115


In [45]:
y_pred = gbm.predict(X_test)
y_pred

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


array([6.57487189e+00, 1.50892649e+00, 1.97598126e+01, ...,
       2.74291907e+03, 3.60251598e+01, 2.23067406e+01])

In [46]:
data_test['id']
output = pd.DataFrame(index=data_test['id'])
output['sales'] = y_pred
output['sales'] = output['sales'].clip(0)

In [47]:
output.to_csv('submission.csv')